In [1]:
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 40)
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_pickle("../data/events/df_events 2025-01-10_16-41-46.pkl")

In [4]:
columns_order = ['StartDate', 'StartTime',  "EndDate", "EndTime", 'StartDateTime', "EndDateTime"]
for col in df.columns:
    if col not in columns_order:
        columns_order.append(col)


In [5]:
# reorder columns 
df = df[columns_order]  

In [6]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)



# preprocess columns

In [7]:
df["Street_orig"] = df["Street"]

In [ ]:
df.City.value_counts()


In [9]:
df.City = df.City.str.replace("im Breisgau", "")
df.City = df.City.str.replace("im breisgau", "")
df.City = df.City.str.strip()

In [10]:
df.rename(columns = {
    "StartTimeIsAssumed": "StartTimeIsUnknown",
}, inplace=True)

In [11]:
df.Street = df.Street.str.replace("Dresden, Germany", "")
df.Street = df.Street.str.replace("Dresden, Alemania", "")

In [ ]:
print(len(df[df.lat.isna()]))

In [ ]:
flt =( df.lat.isna()) & (~df.Street.isna())
print(len(df))
print(len(df[flt]))
df[flt]

In [14]:
#df[df.Street.str.contains("Theaterplatz", na=False)]

In [15]:
df.Street = df.Street.str.lower()

In [16]:
df.Street = df.Street.str.replace("str.", "straße")

In [17]:
df.Street = df.Street.str.replace(",", "")

In [ ]:
df.Street.value_counts()

In [19]:
#flt = df.HouseNumber.str.contains("[a-zA-Z]", na=	False)
#df.loc[flt, "HouseNumber"].value_counts()

In [ ]:
df.head()

In [21]:
#flt = (~df.Street.isna()) & (df.HouseNumber.isna())  & (df.StreetHouseNumber.isna())
#print(len(df[flt]))
#df[flt]

In [22]:
df.Street = df.Street.str.replace("\d{5}", "", regex=True)
df.HouseNumber = df.HouseNumber.str.replace("\d{5}", "", regex=True)

In [ ]:
for pat in [ "(\d+)(\s)+[a-z]", "(\d+)[a-z]", "(\d+)\-(\d+)", "(\d+)"]:
    flt = (df.Street.str.contains(pat, na=False, regex=True)) & (df.HouseNumber.isna())
    values = df.loc[flt].Street.str.extract(pat, expand=False)
    df.loc[flt, "HouseNumber"] = values
    df.Street = df.Street.str.replace(pat, "", regex=True)


In [ ]:
df.Street.value_counts()

In [25]:
df.Street = df.Street.str.replace("  ", " ", regex=False)
df.Street = df.Street.str.replace("/", "", regex=False)
df.Street = df.Street.str.replace(" - ", "", regex=False)
df.Street = df.Street.str.replace("pl.", "platz", regex=False)


In [ ]:
flt = df.Street.str.contains("altplauen ufgang", na=False, regex=True)
df.loc[flt, ["Street", "Street_orig"]]

In [27]:
df.Street = df.Street.str.strip()

In [ ]:
sorted(df.Street.dropna().unique().tolist())

# interpolate coordinated from own values 

## city street house number

In [ ]:
print(len(df[df.lat.isna()]))

In [ ]:
pd.Series([np.nan, 3, 4]).mean()

In [31]:
df["new_lat"] = df.groupby(['City', 'Street', 'HouseNumber'])['lat'].transform(lambda x: x.fillna(x.mean()))

In [32]:
#df.loc[30, ['City', 'Street', 'HouseNumber', "lat", "new_lat"]]

In [ ]:
flt =( df.lat.isna()) & (~df.new_lat.isna())
df.loc[flt, ["lat", "new_lat"]]

In [34]:
flt = df.lat.isna() 
df.loc[flt, "lat"] = df.loc[flt, "new_lat"]

In [ ]:
print(len(df[df.lat.isna()]))

In [36]:
df_empty_lat =df[df.lat.isna()]

## from meslis using StreetHouseNumber

In [37]:
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Dresden, Alemania", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Dresden, Germany", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("Freiburg im Breisgau", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace("\d{5}", "", regex=True)
df.StreetHouseNumber = df.StreetHouseNumber.str.replace(",", "", regex=False)
df.StreetHouseNumber = df.StreetHouseNumber.str.strip()

In [38]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
df.to_pickle(f"../data/events/df_events {time}.pkl")
df.to_csv(f"../data/events/df_events {time}.csv")
